In [ ]:
## import packages
import os, cv2
import numpy as np 
import pandas as pd 
import math
import random, tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import PIL
import PIL.Image

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import math
from tensorflow.keras import layers
from keras.models import Sequential, Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import mobilenet
from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

In [ ]:
# Import functions from Kaggle example code:

# helper function for data visualization
def visualize(**images):
    """
    Plot images in one row
    """
    n_images = len(images)
    plt.figure(figsize=(16,16))
    for idx, (name, image) in enumerate(images.items()):
        plt.subplot(1, n_images, idx + 1)
        # plt.xticks([]); 
        # plt.yticks([])
        # get title from the parameter names
        plt.title(name.replace('_',' ').title(), fontsize=20)
        
        if type(image) == str:
            image = mpimg.imread(image)
        
        plt.imshow(image)
    plt.show()

In [ ]:
## Create path to data

# Gianluca's Path:
#path = "/data"

## Johnathan's Path:
# path = "/content/drive/MyDrive/Grad School/Penn_MUSA/Spring2022/650_RemoteSensing/LULC_FinalProject/data"
## Local path
path = os.getcwd()


## Define path to the data
data_path = "{}".format(path)

In [ ]:
## Classes and their respective pixel values
class_dict = pd.read_csv('{}/class_dict.csv'.format(path))
class_names = class_dict['name'].values.tolist()
class_rgb_vals = class_dict[['r', 'g', 'b']].values.tolist()

# Useful to shortlist specific classes in datasets with large number of classes
select_classes = class_names

# Get RGB values of required classes
select_class_indices = [class_names.index(cls.lower()) for cls in select_classes]
select_class_rgb_values =  np.array(class_rgb_vals)[select_class_indices]

print('Selected classes and their corresponding RGB values in labels:')
print('Class Names: ', class_names)
print('Class RGB values: ', class_rgb_vals)

In [ ]:
# Load metadata file - contains relative paths for images and masks
metadata = pd.read_csv('{}/metadata.csv'.format(path))

# Preparing metadata for use
metadata = metadata[metadata['split']=='train'] # Filter out images that do not have masks (those images are part of the challenge set)
metadata = metadata[['image_id', 'sat_image_path', 'mask_path']] # Remove the image status column

# set paths to absolute paths rather than relative paths:
metadata['sat_image_path'] = metadata['sat_image_path'].apply(lambda img_pth: os.path.join(path, img_pth))
metadata['mask_path'] = metadata['mask_path'].apply(lambda img_pth: os.path.join(path, img_pth))

In [ ]:
metadata.sort_values('image_id')

In [ ]:
img_Loc = metadata['sat_image_path'][245]
mask_Loc = metadata['mask_path'][245]
visualize(Image = (np.asarray(PIL.Image.open(img_Loc))/255), Mask = mask_Loc)

In [ ]:
img = PIL.Image.open(mask_Loc)
uni, freq = np.unique(np.asarray(img), return_counts=True)

print('Unique Vals: {}'.format(uni))
print('Freq of vals: {}'.format(freq))

In [ ]:
# Indentify the number of satellite images and corresponding masks
numSatImg = len(metadata['sat_image_path'])
numMask = len(metadata['mask_path'])
print("Number of original satellite images: " + str(numSatImg))
print("Number of original masks: " + str(numMask))

# Open example image
img = PIL.Image.open(metadata['sat_image_path'][0])
# Get dimensionality of that image
numPixels = np.asarray(img).shape
print('Number of pixels in each image: {}'.format(numPixels))

# Initialize arrays for cropped sat images and masks
# First we will need to trim the 2448 x 2448 image to 2048 x 2048 image so that it can be used with U-net 
# which needs to have images with sizes divisible by 32
# We will then crop each image down to sections of 128 x 128 x 3, which produces 16 cropped images per original sat image.
# If you want to change the size of the cropped image, change the denominator for cropimg width / height below
cropImg_height = int((numPixels[0]-400) / 16) # 2048 / 16 = 128
cropImg_width = int((numPixels[1]-400) / 16) # 2048 / 16 = 128


X = np.zeros([(numSatImg * 256), cropImg_height, cropImg_width, numPixels[2]], dtype='uint8')
y = np.zeros([(numMask * 256), cropImg_height, cropImg_width, numPixels[2]], dtype='uint8')
print('Shape of cropped sat image dataset: {}'.format(X.shape))
print('Shape of cropped mask dataset: {}'.format(y.shape))

In [ ]:
# Extract paths from pandas df to np array for iteration purposes - don't want to use iterrows b/c its slow!
Xrows = np.asarray(metadata['sat_image_path'])
yrows = np.asarray(metadata['mask_path'])


# Step 0: Trim images from 2448x2448 to 2048x24048 to make the image size divisible by 32 for Unet purposes
# Step 1: Iterate through image & corresponding mask paths and read images into memory
# Step 2: iterate through original img matrix and crop to predefined crop height & width
# Step 3: Save cropped matrix to working dataset

cropImgIdx = 0

for i in range(0, len(Xrows)):
  img = np.asarray(cv2.imread(Xrows[i]))
  mask = np.asarray(cv2.imread(yrows[i]))
  
  # Trim image to [2048,2048,3] by trimming extra 200px off from the border
  img = np.array(img[200:2248, 200:2248, :])

  # Iterate through each row
  for r in range(0, img.shape[0], cropImg_height):
    # Iterate through each column
    for c in range(0, img.shape[1], cropImg_width):

      # Slice mask by cropping window first
      # That way we can check if if we're going to use the image or not
      newMask = np.array(mask[r:r+cropImg_height, c:c+cropImg_width, :])
      newMask = cv2.cvtColor(newMask, cv2.COLOR_BGR2RGB)
      
      # Convert mask to grayscale to find distribution of classes
      grayMask = cv2.cvtColor(newMask, cv2.COLOR_BGR2GRAY)
      # Get frequency of each classification in the cropped mask
      unique, frequency = np.unique(grayMask, return_counts= True)
      frequency = frequency / (len(grayMask.flatten()))

      # Check if any classes are represent 99% of image
      # If that is the case, throw it out
      if (frequency >= 0.99).any():
        continue

      # Crop image if we make it past the majority class checker
      newImg = np.array(img[r:r+cropImg_height, c:c+cropImg_width, :])
      newImg = cv2.cvtColor(newImg, cv2.COLOR_BGR2RGB)

      X[cropImgIdx,:] = newImg
      y[cropImgIdx,:] = newMask

      cropImgIdx += 1

print('There were {} images cropped from {} original images'.format((cropImgIdx-1), numSatImg))

In [ ]:
cropImgIdx -= 1

In [ ]:
X = X[0:cropImgIdx, :,:,:]
y = y[0:cropImgIdx, :,:,:]

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
ogImg = Xrows[0]
ogmask = yrows[0]
cropImg = X[0]
cropMsk = y[0]

visualize(original_Image = ogImg, original_Mask = ogmask)
visualize(crop_Image = cropImg, crop_Mask = cropMsk)

# U-Net model building

In [ ]:
!pip install keras-unet

from keras_unet.models import vanilla_unet, custom_unet

In [ ]:
## Shuffle DataFrame
## For function origin, check this stack overflow: https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison
def unison_shuffled(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

X_shuf, y_shuf = unison_shuffled(X, y)

In [ ]:
## Downsample so that we don't break our computers
X_down = X_shuf[0:1000, :,:,:]
y_down = y_shuf[0:1000, :,:,:]
print(X_down.shape)
print(y_down.shape)


In [ ]:
print(X_down.max(), y_down.max())

In [ ]:
print(X_down.shape, y_down.shape)

In [ ]:
x = np.asarray(X_down, dtype=np.float32)/255
y = np.asarray(y_down, dtype=np.float32)/255

In [ ]:
print(x.max(), y.max())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test: ", y_test.shape)

In [ ]:
mask_train = to_categorical(mask_train, 4)
mask_train = mask_train.reshape((634,64,64,4)) 

In [ ]:
print("y_train: ", y_train.shape)

In [ ]:
from keras_unet.models import custom_unet

input_shape = X_train[0].shape

model = custom_unet(
    input_shape,
    use_batch_norm=False,
    num_classes=7,
    filters=64,
    dropout=0.2,
    output_activation='sigmoid'
)

In [ ]:
model.summary()

# Compile + train

In [ ]:
from keras.callbacks import ModelCheckpoint


model_filename = 'segm_model_v0.h5'
callback_checkpoint = ModelCheckpoint(
    model_filename, 
    verbose=1, 
    monitor='val_loss', 
    save_best_only=True,
)

In [ ]:
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.losses import jaccard_distance

model.compile(
    optimizer=Adam(), 
    #optimizer=SGD(lr=0.01, momentum=0.99),
    #loss='binary_crossentropy',
    loss=jaccard_distance,
    metrics=[iou, iou_thresholded]
)

In [ ]:
history = model.fit(X_train, y_train, batch_size = 32, epochs = 50, steps_per_epoch=100)